In [2]:
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, mean_squared_error

In [3]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/패캠강의/fastcampus_recsys_2301/Fastcampus_Codes/02_ML_based_algorithms/')

Mounted at /content/drive


In [13]:
class FM:
    def __init__(self, n_factors=10, learning_rate=0.01, n_epochs=10):
        self.n_factors = n_factors
        self.learning_rate = learning_rate
        self.n_epochs = n_epochs

    def fit(self, X, y, X_val, y_val):
        # label encoder는 1부터 카운트하기 때문에, nunique를 지정해 주기 위해 1을 더해 줌
        self.n_users = int(np.max(X[:, 0]) + 1)
        self.n_items = int(np.max(X[:, 1]) + 1)
        
        # 실제 y label의 평균을 global bias로 설정
        self.global_bias = np.mean(y)
        
        # 유저/아이템 피처별 bias를 0으로 셋팅
        self.user_bias = np.zeros(self.n_users)
        self.item_bias = np.zeros(self.n_items)
        
        # 평균 0, std 0.1인 정규분포에서 유저 및 아이템 팩터 초기화, 각 대상 별 n_factors개의 팩터를 할당
        self.user_factors = np.random.normal(scale=0.1, size=(self.n_users, self.n_factors))
        self.item_factors = np.random.normal(scale=0.1, size=(self.n_items, self.n_factors))

        for epoch in tqdm(range(self.n_epochs), desc='Training epoch'):
            
            # 각 epoch에 따라 다음을 수행
            for i in range(X.shape[0]):
                
                # 학습셋 내의 user-item pair를 대상으로 다음을 반복
                user, item = int(X[i, 0]), int(X[i, 1])
                
                # 예측 값 생성
                prediction = self.predict(user, item)
                
                # 실제 값과 예측 값을 대조함으로써 error 계산 -> loss function 계산에 사용 
                e = (y[i] - prediction)  
                
                # Loss function을 bias term에 대해 미분 -> (e - bias)로 계산
                self.user_bias[user] += self.learning_rate * (e - self.user_bias[user])
                self.item_bias[item] += self.learning_rate * (e - self.item_bias[item])
                
                # pairwise term의 Loss function을 latent factor에 대해 미분 -> (e * 상대 factor - 타겟 factor)로 계산
                self.user_factors[user] += self.learning_rate * (e * self.item_factors[item] - self.user_factors[user])
                self.item_factors[item] += self.learning_rate * (e * self.user_factors[user] - self.item_factors[item])

            # 전체 로스 계산
            loss = 0
            for i in range(X.shape[0]):
                user, item = int(X[i, 0]), int(X[i, 1])
                prediction = self.predict(user, item)
                loss += (y[i] - prediction) ** 2

            # validation pair에 대해 metrics 계산
            y_pred = [self.predict(user, item) for user, item in X_val]
            auc_score = roc_auc_score(y_val, y_pred)
            rmse = np.sqrt(mean_squared_error(y_val, y_pred))

            print(f'Epoch {epoch + 1}/{self.n_epochs} - loss: {loss / X.shape[0]} - val_auc: {auc_score} - val_rmse: {rmse}')

        return self

    def predict(self, user, item):
        # global bias와 user/item bias를 각각 더해주고, item/user factor를 내적해 준 값을 더해 최종 예측을 생성
        prediction = (self.global_bias +
                      self.user_bias[user] +
                      self.item_bias[item] +
                      np.dot(self.item_factors[item], self.user_factors[user]))
        return prediction

In [32]:
# 데이터셋 로드
data_path = '../data/anime/'

anime = pd.read_csv(data_path+'anime.csv')
rating = pd.read_csv(data_path+'rating.csv')

In [33]:
rating_sampled = rating.sample(100_000, random_state=42)

In [34]:
# 데이터 전처리 
merged = pd.merge(anime, rating_sampled, on='anime_id', suffixes= ['_anime', '_user'])
merged = merged.dropna()
merged = merged[['anime_id', 'name', 'genre', 'type', 'episodes', 'rating_user', 'user_id']]

# user/item id 인코딩 
user_enc = LabelEncoder()
anime_enc = LabelEncoder()
merged['user'] = user_enc.fit_transform(merged['user_id'].values)
merged['anime'] = anime_enc.fit_transform(merged['anime_id'].values)

Training epoch:  10%|█         | 1/10 [00:02<00:26,  2.93s/it]

Epoch 1/10 - loss: 0.1723596429942134 - val_auc: 0.6728830165111832 - val_rmse: 0.4216138219265369


Training epoch:  20%|██        | 2/10 [00:05<00:20,  2.60s/it]

Epoch 2/10 - loss: 0.16259868204420502 - val_auc: 0.7245439470436138 - val_rmse: 0.41529465900901613


Training epoch:  30%|███       | 3/10 [00:07<00:17,  2.49s/it]

...

Epoch 8/10 - loss: 0.13073918700787202 - val_auc: 0.7992363157193408 - val_rmse: 0.39413909338128655


Training epoch:  90%|█████████ | 9/10 [00:27<00:03,  3.21s/it]

Epoch 9/10 - loss: 0.12746522711951758 - val_auc: 0.8030466208404055 - val_rmse: 0.39195130124153416


Training epoch: 100%|██████████| 10/10 [00:30<00:00,  3.10s/it]

Epoch 10/10 - loss: 0.12451351804923798 - val_auc: 0.8060900247795616 - val_rmse: 0.3899851320949919
sparsity:  0.000459066495809462


In [35]:
# 6 이상의 평점을 갖는 경우 1로, 그 외에는 0으로 인코딩
merged['rating'] = [1 if x>=6 else 0 for x in merged['rating_user']]

# id 정보만을 사용
X = merged[['user', 'anime']].values
y = merged['rating'].values


In [36]:
X[:5]

array([[17428,  5652],
       [ 9387,  5652],
       [21969,  5652],
       [  698,  5652],
       [ 2286,  5652]])

In [37]:
y[:5]

array([1, 1, 1, 1, 1])

In [39]:
# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 인스턴스 생성 및 학습 
fm = FM(n_factors=10, learning_rate=0.01, n_epochs=10)
fm.fit(X_train, y_train, X_val, y_val)

n_users = int(np.max(X_train[:, 0]) + 1)
n_items = int(np.max(X_train[:, 1]) + 1)

print("sparsity: ", rating_sampled.shape[0]/(n_users * n_items))

Training epoch:  10%|█         | 1/10 [00:02<00:20,  2.26s/it]

Epoch 1/10 - loss: 0.17241553843581012 - val_auc: 0.6752849245777984 - val_rmse: 0.4214228474980061


Training epoch:  20%|██        | 2/10 [00:04<00:18,  2.31s/it]

Epoch 2/10 - loss: 0.16263964857953409 - val_auc: 0.7263915520673637 - val_rmse: 0.41515499206313866


Training epoch:  30%|███       | 3/10 [00:07<00:16,  2.39s/it]

...

Epoch 8/10 - loss: 0.13073986809525412 - val_auc: 0.7991617482175808 - val_rmse: 0.3941062646657229


Training epoch:  90%|█████████ | 9/10 [00:24<00:02,  2.83s/it]

Epoch 9/10 - loss: 0.12746492809657636 - val_auc: 0.8028540778597781 - val_rmse: 0.39192534703758874


Training epoch: 100%|██████████| 10/10 [00:27<00:00,  2.77s/it]

Epoch 10/10 - loss: 0.12451274389016548 - val_auc: 0.8058521086854988 - val_rmse: 0.3899649840270636
sparsity:  0.000459066495809462


In [29]:
pd.DataFrame(X_train)[0].value_counts().mean()

2.335503459550988

In [ ]:
# step by step

In [17]:
def predict(user, item):
    # global bias와 user/item bias를 각각 더해주고, item/user factor를 내적해 준 값을 더해 최종 예측을 생성
    prediction = (global_bias +
                    user_bias[user] +
                    item_bias[item] +
                    np.dot(item_factors[item], user_factors[user]))
    return prediction

In [18]:
n_epochs = 1
n_factors = 8
learning_rate = 0.01

# label encoder는 1부터 카운트하기 때문에, nunique를 지정해 주기 위해 1을 더해 줌
n_users = int(np.max(X[:, 0]) + 1)
n_items = int(np.max(X[:, 1]) + 1)

Training epoch: 100%|██████████| 1/1 [00:03<00:00,  3.31s/it]

Epoch 1/1 - loss: 0.17110637498655906 - val_auc: 0.8023054817881665 - val_rmse: 0.41271441474929693


In [20]:
print(n_users, n_items)

37838 5757


In [ ]:
# 실제 y label의 평균을 global bias로 설정
global_bias = np.mean(y)

In [21]:
global_bias

0.751085021700434

In [40]:
# 유저/아이템 피처별 bias를 0으로 셋팅
user_bias = np.zeros(n_users)
item_bias = np.zeros(n_items)

# 평균 0, std 0.1인 정규분포에서 유저 및 아이템 팩터 초기화, 각 대상 별 n_factors개의 팩터를 할당
user_factors = np.random.normal(scale=0.1, size=(n_users, n_factors))
item_factors = np.random.normal(scale=0.1, size=(n_items, n_factors))

In [43]:
user_factors[:10]

array([[ 0.1092795 , -0.12088772,  0.02212493,  0.09984646, -0.07448534,
        -0.05117256, -0.0320646 ,  0.06428975],
       [-0.2217324 , -0.17362378,  0.00908396,  0.11450284, -0.07122338,
        -0.02931193, -0.11545684, -0.02594664],
       [-0.03000867, -0.12746609,  0.05367946, -0.05765581, -0.17685408,
         0.01809996, -0.23176421,  0.08548527],
       [ 0.13715115,  0.18364744, -0.06137639, -0.09898509,  0.03221877,
         0.01196739, -0.07924032,  0.0236374 ],
       [ 0.19615906, -0.14759854,  0.09061463, -0.09427315, -0.00926252,
        -0.11873279,  0.00775792,  0.13320269],
       [ 0.02478118, -0.0143987 , -0.06846221, -0.13220904, -0.24415747,
         0.04777324, -0.03819582,  0.1954017 ],
       [ 0.080346  ,  0.00177874, -0.09574243, -0.0149212 ,  0.04971864,
        -0.08363625, -0.01315097, -0.04980055],
       [-0.14009802,  0.00475749,  0.17803071,  0.16373105, -0.04011767,
         0.01336875,  0.19974464,  0.01170048],
       [ 0.14619131,  0.05219394

In [ ]:
for epoch in tqdm(range(n_epochs), desc='Training epoch'):
    
    # 각 epoch에 따라 다음을 수행
    for i in range(X.shape[0]):
        
        # 학습셋 내의 user-item pair를 대상으로 다음을 반복
        user, item = int(X[i, 0]), int(X[i, 1])
        
        # 예측 값 생성
        prediction = predict(user, item)
        
        # 실제 값과 예측 값을 대조함으로써 error 계산 -> loss function 계산에 사용 
        e = (y[i] - prediction)  
        
        # Loss function을 bias term에 대해 미분 -> (e - bias)로 계산
        user_bias[user] += learning_rate * (e - user_bias[user])
        item_bias[item] += learning_rate * (e - item_bias[item])
        
        # pairwise term의 Loss function을 latent factor에 대해 미분 -> (e * 상대 factor - 타겟 factor)로 계산
        user_factors[user] += learning_rate * (e * item_factors[item] - user_factors[user])
        item_factors[item] += learning_rate * (e * user_factors[user] - item_factors[item])
        break

    # 전체 로스 계산
    loss = 0
    for i in range(X.shape[0]):
        user, item = int(X[i, 0]), int(X[i, 1])
        prediction = predict(user, item)
        loss += (y[i] - prediction) ** 2

    # validation pair에 대해 metric 계산
    y_pred = [predict(user, item) for user, item in X_val]
    auc_score = roc_auc_score(y_val, y_pred)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))

    print(f'Epoch {epoch + 1}/{n_epochs} - loss: {loss / X.shape[0]} - val_auc: {auc_score} - val_rmse: {rmse}')

In [44]:
rating_sampled = rating.sample(1_000_000, random_state=42)

# Preprocess the data
merged = pd.merge(anime, rating_sampled, on='anime_id', suffixes= ['_anime', '_user'])
merged = merged.dropna()
merged = merged[['anime_id', 'name', 'genre', 'type', 'episodes', 'rating_user', 'user_id']]

# Create features from user and anime
user_enc = LabelEncoder()
anime_enc = LabelEncoder()
merged['user'] = user_enc.fit_transform(merged['user_id'].values)
merged['anime'] = anime_enc.fit_transform(merged['anime_id'].values)

# Transform ratings to binary
merged['rating'] = [1 if x>=6 else 0 for x in merged['rating_user']]

# Prepare the data
X = merged[['user', 'anime']].values
y = merged['rating'].values


# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Create and train the FM model
fm = FM(n_factors=10, learning_rate=0.01, n_epochs=10)
fm.fit(X_train, y_train, X_val, y_val)

n_users = int(np.max(X_train[:, 0]) + 1)
n_items = int(np.max(X_train[:, 1]) + 1)

print("sparsity: ", rating_sampled.shape[0]/(n_users * n_items))

In [45]:
pd.DataFrame(X_train)[0].value_counts().mean()

12.611678648001009

In [30]:
rating_sampled = rating.sample(10_000, random_state=42)

# Preprocess the data
merged = pd.merge(anime, rating_sampled, on='anime_id', suffixes= ['_anime', '_user'])
merged = merged.dropna()
merged = merged[['anime_id', 'name', 'genre', 'type', 'episodes', 'rating_user', 'user_id']]

# Create features from user and anime
user_enc = LabelEncoder()
anime_enc = LabelEncoder()
merged['user'] = user_enc.fit_transform(merged['user_id'].values)
merged['anime'] = anime_enc.fit_transform(merged['anime_id'].values)

# Transform ratings to binary
merged['rating'] = [1 if x>=6 else 0 for x in merged['rating_user']]

# Prepare the data
X = merged[['user', 'anime']].values
y = merged['rating'].values


# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the FM model
fm = FM(n_factors=20, learning_rate=0.01, n_epochs=30)
fm.fit(X_train, y_train, X_val, y_val)


n_users = int(np.max(X_train[:, 0]) + 1)
n_items = int(np.max(X_train[:, 1]) + 1)

print("sparsity: ", rating_sampled.shape[0]/(n_users * n_items))

Training epoch:   3%|▎         | 1/30 [00:00<00:06,  4.18it/s]

Epoch 1/30 - loss: 0.1788476139407178 - val_auc: 0.4729431278109152 - val_rmse: 0.44674377877067534


Training epoch:   7%|▋         | 2/30 [00:00<00:07,  3.80it/s]

Epoch 2/30 - loss: 0.17035288089875697 - val_auc: 0.4985614493955536 - val_rmse: 0.44530924020434215


Training epoch:  10%|█         | 3/30 [00:00<00:07,  3.83it/s]

...

Epoch 28/30 - loss: 0.08674327482726338 - val_auc: 0.6123045516889414 - val_rmse: 0.4349965083449746


Training epoch:  97%|█████████▋| 29/30 [00:10<00:00,  3.22it/s]

Epoch 29/30 - loss: 0.08538284399448744 - val_auc: 0.6128544544033684 - val_rmse: 0.4348671841030241


Training epoch: 100%|██████████| 30/30 [00:10<00:00,  2.91it/s]

Epoch 30/30 - loss: 0.08407882975310256 - val_auc: 0.6134120123760007 - val_rmse: 0.4347441682509361
sparsity:  0.00042599559750589803


In [31]:
pd.DataFrame(X_train)[0].value_counts().mean()

1.1484352569623888